# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Starter Code

## Part A | RDBMS Databases and `SQLite`

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-04-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [26]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 100
;
''', con = db)

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,2121978635,"829 Folsom St UNIT 906, San Francisco, CA",37781429,-122401860,0.0,...,557.0,sqft,NaN,None,2010.0
1,89239580,"690 Market St UNIT 1705, San Francisco, CA",37788246,-122403198,0.0,...,1050.0,sqft,NaN,None,2007.0
2,15131782,"401 Grand View Ave APT 3, San Francisco, CA",37752157,-122442356,0.0,...,937.0,sqft,NaN,None,1983.0
3,15179502,"250 Concord St, San Francisco, CA",37710141,-122442063,0.0,...,1574.0,sqft,1947.0,sqft,1959.0
4,52266124,"88 King St APT 317, San Francisco, CA",37780630,-122389635,0.0,...,1205.0,sqft,NaN,None,2000.0
...,...,...,...,...,...,...,...,...,...,...,...
95,15161805,"183 Bocana St, San Francisco, CA",37740976,-122417224,0.0,...,1275.0,sqft,2500.0,sqft,1922.0
96,15182023,"1381 Sanchez St, San Francisco, CA",37747076,-122428941,0.0,...,2000.0,sqft,1999.0,sqft,1900.0
97,15081147,"2955 Pacific Ave, San Francisco, CA",37791900,-122443975,0.0,...,2160.0,sqft,NaN,None,1995.0
98,15143800,"30 Abbey St, San Francisco, CA",37763451,-122427635,0.0,...,1115.0,sqft,NaN,None,1900.0


### `SELECT`

In [28]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,id,sale_price,sale_price_unit
0,15165953,650000.00,$
1,80749447,1.15,$M
2,15155751,665000.00,$
3,15143887,2.10,$M
4,15117639,1.35,$M
...,...,...,...
995,69819708,731000.00,$
996,15076156,5.53,$M
997,119685619,625000.00,$
998,15113584,895000.00,$


### `WHERE`

In [29]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,80749447,1.15
1,15143887,2.10
2,15117639,1.35
3,80743040,1.48
4,15134909,1.09
...,...,...
570,2101028916,2.45
571,15091950,1.75
572,89237766,1.16
573,15197693,1.36


> ### Here's let's convert the unit of `sale_price` from \$ to \$M

In [31]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
420,119685503,0.750
421,69819708,0.731
422,119685619,0.625
423,15113584,0.895


### `UNION`

> ### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [32]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `WITH`

In [33]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `JOIN`

In [34]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `WITH` (take 2)

In [35]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM baths_normalized_sale_prices
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `GROUP BY` and `COUNT`

> ### Let's compute the number of properties based on their number of bedrooms

In [37]:
pd.io.sql.read_sql(
'''
SELECT beds, COUNT() AS count
    FROM properties
    GROUP BY beds
;
''', con = db)

,beds,count
0,NaN,164
1,1.0,142
2,2.0,316
3,3.0,226
4,4.0,103
5,5.0,28
6,6.0,9
7,7.0,6
8,8.0,3
9,9.0,3


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bathrooms

In [39]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT baths, AVG(sale_price) AS mean_sale_price
    FROM baths_normalized_sale_prices
    GROUP BY baths
;
''', con = db)

,baths,mean_sale_price
0,NaN,1.739026
1,1.00,0.987656
2,1.10,1.420000
3,1.25,1.600000
4,1.50,1.223378
...,...,...
16,6.50,16.000000
17,7.00,0.999000
18,7.50,5.530000
19,8.00,13.100000


Note: `baths = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [60]:
pd.io.sql.read_sql(
'''

WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

studio_price AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        LEFT JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id
        WHERE properties.is_a_studio = 1)

SELECT beds, AVG(sale_price) AS mean_sale_price
    FROM studio_price
    GROUP BY beds

''', con = db)

,beds,mean_sale_price
0,None,1.464034


> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [71]:
pd.io.sql.read_sql(
'''

SELECT a.id AS id_a, b.id AS id_a, a.address AS address_a, b.address AS address_b
    FROM properties AS a
    JOIN properties AS b ON b.latitude = a.latitude AND b.longitude = a.longitude
    WHERE a.address <> b.address
    
;
''', con = db)

,id_a,id_a,address_a,address_b
0,52266124,53153575,"88 King St APT 317, San Francisco, CA","88 King St APT 419, San Francisco, CA"
1,119685526,119685188,"1902 Filbert St, San Francisco, CA","1902A Filbert St, San Francisco, CA"
2,15076711,15076699,"1 Daniel Burnham Ct APT 1404, San Francisco, CA","1 Daniel Burnham Ct APT 1202, San Francisco, CA"
3,15068514,15068551,"1177 California St APT 702, San Francisco, CA","1177 California St APT 501, San Francisco, CA"
4,15068514,15068580,"1177 California St APT 702, San Francisco, CA","1177 California St APT 423, San Francisco, CA"
...,...,...,...,...
209,60825591,61288454,"77 Dow Pl APT 500, San Francisco, CA","77 Dow Pl APT 507, San Francisco, CA"
210,67395015,63107226,"8400 Oceanview Ter APT 218, San Francisco, CA","3981 Alemany Blvd APT 206, San Francisco, CA"
211,67395015,67394898,"8400 Oceanview Ter APT 218, San Francisco, CA","8300 Oceanview Ter APT 306, San Francisco, CA"
212,63197318,63197354,"765 Market St APT 27B, San Francisco, CA","765 Market St APT 31H, San Francisco, CA"
